In [8]:
import numpy as np
import pandas as pd
import pickle
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as Log
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict, GridSearchCV

#### Load the data

In [2]:
# read the data with the non-text features
patents = pd.read_pickle("patent_data/nontext_features.p")

In [3]:
print patents.shape
patents.head()

(12030, 16)


,id,publication_year,B,C,D,E,F,G,H,num_applications,num_patent_citations,num_nonpatent_citations,num_claims,num_similar_doc,num_authors,payment_times
0,US6699658B1,2004,0,1,0,0,0,0,0,5,28,34,42,1,4,3
1,US6699724B1,2004,0,0,0,0,0,1,0,32,47,44,25,0,4,3
2,US6690816B2,2004,0,0,0,0,0,1,0,8,9,0,32,1,4,1
3,US6711436B1,2004,0,0,0,0,0,0,0,4,105,109,45,7,1,3
4,US6711432B1,2004,0,0,0,0,0,0,0,7,15,114,44,3,4,3


#### Format the data

In [4]:
# reformat the response variable into binary
y_data = np.zeros(patents.shape[0])
y_data[patents['payment_times'].values >= 2] = 1

print "Percentage of patents with > 1 maintenance fee payments: ", np.mean(y_data)

Percentage of patents with > 1 maintenance fee payments:  0.62859517872


In [5]:
# predictors
x_data = patents.drop(['id', 'payment_times'], axis = 1).values
x_data.shape

(12030, 14)

In [6]:
# split train and test sets
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, 
                                                    test_size = 0.2, 
                                                    random_state = 123)

print "Dataset dimensions:"
print "x_train: ", x_train.shape
print "x_test: ", x_test.shape
print "y_train: ", y_train.shape
print "y_test: ", y_test.shape

Dataset dimensions:
x_train:  (9624, 14)
x_test:  (2406, 14)
y_train:  (9624,)
y_test:  (2406,)


### Model training
#### Basic Logistic Regression

In [9]:
### use logistic regression

# call the model function
model = Log()
# parameter tuning
c =  np.logspace(-5, 5, 11)

# use grid search with 5-fold CV
grid_model = GridSearchCV(model, param_grid = {'C': c}, cv  = 5, scoring = 'accuracy')
# fit on the data
grid_model = grid_model.fit(x_train, y_train) 

In [10]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.628325020781
Best parameter:  {'C': 100000.0}


In [11]:
# check confusion matrix
best_log = grid_model.best_estimator_
y_pred = cross_val_predict(best_log, x_train, y_train, cv = 5)

metrics.confusion_matrix(y_train, y_pred)

array([[ 114, 3485],
       [  92, 5933]])

#### Logistic Regression with Balanced Weight

In [12]:
# call the model function with balanced weight
model = Log(class_weight='balanced')
# parameter tuning
c =  np.logspace(-5, 5, 11)

# use grid search with 5-fold CV
grid_model = GridSearchCV(model, param_grid = {'C': c}, cv  = 5, scoring = 'accuracy')
# fit on the data
grid_model = grid_model.fit(x_train, y_train) 

In [13]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

# check confusion matrix
best_log = grid_model.best_estimator_
y_pred = cross_val_predict(best_log, x_train, y_train, cv = 5)

metrics.confusion_matrix(y_train, y_pred)

Best accuracy: 0.549875311721
Best parameter:  {'C': 10.0}


array([[2106, 1493],
       [2839, 3186]])

#### Random Forest

In [14]:
### tune random forest

model = RandomForestClassifier(n_estimators = 100)

# tune max_features
param_space = np.arange(2, 15, 2)

grid_model = GridSearchCV(model, n_jobs = 4, 
                          param_grid = {'max_features': param_space}, 
                          cv  = 5, scoring = 'accuracy')
# fit on the data
grid_model = grid_model.fit(x_train, y_train)

In [15]:
# check results
print "Best accuracy:", grid_model.best_score_
print "Best parameter: ", grid_model.best_params_

Best accuracy: 0.613362427265
Best parameter:  {'max_features': 10}


In [16]:
# check confusion matrix
best_rf = grid_model.best_estimator_
y_pred = cross_val_predict(best_rf, x_train, y_train, cv = 5)

metrics.confusion_matrix(y_train, y_pred)

array([[ 953, 2646],
       [1093, 4932]])